### ***Wczytanie danych***
https://www.kaggle.com/datasets/mylesoneill/world-university-rankings

Dane które wybraliśmy dotyczą rankingu uczelni wyższych. Celem analizy jest wyłonić najlepszy uniwersytet w ramach całego świata.

In [ ]:
import numpy as np
import pandas as pd


df = pd.read_csv("cwurData.csv", index_col=False)

df.head()

### ***Opis zmiennych***

In [ ]:
df.info()

###*Podział zmiennych na kategoryczne / numeryczne*

In [ ]:
categorical = [var for var in df.columns if df[var].dtype == 'O']
numerical = [var for var in df.columns if df[var].dtype != 'O']

df[categorical].isnull().sum()

In [ ]:
df[numerical].isnull().sum()

###*Usunięcie rekordów które zawierają niepoprawną wartość w kolumnie 'broad_impact'*

In [ ]:
df = df.dropna(axis=0, subset=['broad_impact'])
df[numerical].isnull().sum()

###*Teraz nasz data-set jest już oczyszczony ze zbędnych rekordów*

In [ ]:
df.info()

###*Oddzielenie kolumny która determinuję pozycję uczelni od reszty zmiennych*

In [ ]:
X = df.drop(['world_rank'], axis=1)
y = df['world_rank']

###*Podział na zbiory treningowy / testowy + Trenowanie modelu*

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Podzial na zbior treningowy i testowy (np. 80% danych do treningu, 20% do testow)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicjalizacja modelu regresji losowego lasu (Random Forest Regressor)
model = RandomForestRegressor(random_state=42)

# Trenowanie modelu na danych treningowych
model.fit(X_train, y_train)

# Przewidywanie na danych testowych
predictions = model.predict(X_test)

# Obliczanie bledu sredniokwadratowego (MSE) jako miary jakosci modelu
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error:", mse)
